# RTML Lab 11: GNNs

Today we explore GNNs, in particular [Graph Convolutional Networks by Kipf and Welling (2017)](https://arxiv.org/abs/1609.02907).

This tutorial is based on the [PyTorch Geometric tutorial for GCN](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html).

## Setup

PyG only seems to work with recent versions of PyTorch, so you may need to upgrade:

In [ ]:
!pip install --upgrade torch
!pip install torch_geometric

Be sure to restart your kernel if you're doing this in Jupyter.

## Our first graph

OK, with that set up, let's use PyG to build a graph:

In [1]:
import torch
from torch_geometric.data import Data

# os.environ['http_proxy'] = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
    
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)


In [3]:
data

Data(x=[3, 1], edge_index=[2, 4])

So we see that this defines a graph with three nodes and two edges. The edge index
is in so-called [COO format](https://pytorch.org/docs/stable/sparse.html#sparse-coo-docs) (a format for representing sparse matrices)
and in this case is a $2\times 2|E|$ tensor.

The indices are indexing the set of nodes, in the range 0 to $N-1$.

The following is equivalent. Note that each undirected edge is denoted by two pairs of indices:

In [ ]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

The tensor `x` is providing the *features* of the nodes in the graph, in this case a scalar for each node. As we know,
node features can be vectors as well, but all nodes must have the same feature dimensionality.

We can check the validity of our graph with the `validate()` method:

In [4]:
data.validate(raise_on_error=True)

True

Try some other attributes and:
- `x`
- `edge_index`
- `edge_attr`
- `num_nodes`
- `num_edges`
- `num_node_features`
- `has_isolated_nodes()`
- `has_self_loops`
- `is_directed`

And you can move a graph to a GPU just like a tensor:

In [5]:
device = torch.device('cuda:0')
data = data.to(device)

## PyG datasets (Cora)

Let's load the Cora dataset:

In [6]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./data/Cora', name='Cora')

Processing...
Done!


Try a few of the dataset's methods:
- `len(dataset)`
- `dataset.num_classes`
- `dataset.num_node_features`
- `dataset[0]`

From this we see that Cora is a single graph.
We will only be doing transduction with this dataset.
After assigning the single graph to a variable such as `data`,
explore its characteristics:
- `data.is_undirected()`
- `data.train_mask.sum().item()`
- `data.val_mask.sum().item()`
- `data.test_mask.sum().item()`

As this graph is mainly used to test semi-supervised learning algorithms, we see that the validation and test data are more numerous than the training data.


## Mini-batches

How can we train on multiple examples in parallel? PyG puts multiple graphs together with block-diagonal adjacency matrices and concatenated features.
See the [tutorial section on mini-batches](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#mini-batches) for more
information.

For a GCN on Cora, however, we don't need fancy data loaders or parallel mini-batches, as we're dealing with just one graph.

## Define a GCN

So let's define a GCN for Cora, then:

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

Moving the model to a GPU and training is the same as for any other PyTorch model. The "data" is treated as a single example:

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

After training, we an evaluate on the test dataset:

In [10]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7950


## Exercises

1. Implement early stopping using the validation set for the Cora GCN.
2. Reproduce the rest of the results from Kipf and Welling with your GCN.
3. Implement the [Graph Attention Network](https://arxiv.org/abs/1710.10903) and reproduce their results on the datasets available via PyG.
